# CARGA DE DATOS

In [27]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv("C:\\Users\\roesc\\Desktop\\hospital-efficiency-dashboard_data\\df_consolidado_final_v2.csv")

# Mostrar las primeras filas del DataFrame
df.head()

,hospital_id,region_id,hospital_name,hospital_alternative_name,latitud,longitud,Consultas,GRDxEgreso,Bienes y servicios,Remuneraciones,Dias Cama Disponibles,Consultas Urgencias,Examenes,Quirofanos,Año
0,101100,15,Hospital Dr. Juan Noé Crevanni (Arica),Hospital Doctor Juan Noé,-18.4827,-70.3126,184208.0,11953.82980,19779705,10982608,113311.0,139557.0,898535.0,178.0,2014
1,102100,1,Hospital Dr. Ernesto Torres Galdames (Iquique),Hospital Iquique,-20.2139,-70.1383,149392.0,13810.48800,27881106,15394204,161383.0,104376.0,756183.0,9.0,2014
2,103100,2,Hospital Dr. Leonardo Guzmán (Antofagasta),Hospital de Antofagasta,-23.6597,-70.3959,154729.0,15613.32800,29969438,15065061,196717.0,55364.0,897022.0,0.0,2014
3,103101,2,Hospital Dr. Carlos Cisternas (Calama),Hospital de Calama,-22.4450,-68.9181,36390.0,6229.00320,11358890,3408115,72833.0,94649.0,403890.0,0.0,2014
4,103102,2,Hospital Dr. Marcos Macuada (Tocopilla),Hospital de Tocopilla,-22.0882,-70.1929,4752.0,746.24524,3524477,1380911,14564.0,32963.0,148503.0,0.0,2014


# PRE PROCESADO

In [28]:
# pasar valores '--' a 0 en columnas 'Bienes y servicios' y 'Remuneraciones'
df['Bienes y servicios'] = df['Bienes y servicios'].replace('--', 0)
df['Remuneraciones'] = df['Remuneraciones'].replace('--', 0)

# pasar columnas de tipo string a tipo float
cols_to_float = ['Bienes y servicios', 'Remuneraciones']
for col in cols_to_float:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)


In [29]:
# contar missing en las columnas usando pandas
missing_counts = df.isnull().sum()
# mostrar los conteos de valores faltantes
print(missing_counts)

hospital_id                    0
region_id                      0
hospital_name                  0
hospital_alternative_name      6
latitud                        0
longitud                       0
Consultas                      0
GRDxEgreso                   134
Bienes y servicios             0
Remuneraciones                 0
Dias Cama Disponibles          8
Consultas Urgencias            0
Examenes                       0
Quirofanos                     0
Año                            0
dtype: int64


In [30]:
# pasar missing a 0
df.fillna(0, inplace=True)

In [31]:
# contar missing en las columnas usando pandas
missing_counts = df.isnull().sum()
# mostrar los conteos de valores faltantes
print(missing_counts)

hospital_id                  0
region_id                    0
hospital_name                0
hospital_alternative_name    0
latitud                      0
longitud                     0
Consultas                    0
GRDxEgreso                   0
Bienes y servicios           0
Remuneraciones               0
Dias Cama Disponibles        0
Consultas Urgencias          0
Examenes                     0
Quirofanos                   0
Año                          0
dtype: int64


# Cálculos

## SFA

In [32]:
df_copy = df.copy()
df_2014 = df_copy[df_copy["Año"] == 2014]

In [39]:
import pandas as pd
import numpy as np
from pysfa import SFA

inputs_cols = ['Remuneraciones', 'Bienes y servicios', 'Dias Cama Disponibles']
outputs_cols = ['Consultas Urgencias']

# seleccionar hospital_id y las columnas de inputs y outputs
df_sample = df_2014[['hospital_id'] + inputs_cols + outputs_cols].sample(n=100, random_state=42)

# solo conservar las filas donde los inputs y outputs son mayores que 0
df_sample = df_sample[(df_sample[inputs_cols] > 0).all(axis=1) & (df_sample[outputs_cols] > 0).all(axis=1)]

x = np.log(df_sample[inputs_cols]).to_numpy()   # aplicar logaritmo a los inputs
y = np.log(df_sample[outputs_cols]).to_numpy()   # aplicar logaritmo a los outputs



# 3. Crear y optimizar SFA
sfa = SFA.SFA(y, x, fun=SFA.FUN_PROD, method=SFA.TE_teJ)
sfa.optimize()
# 4. Añadir eficiencia al DataFrame
df_sample['SFA'] = sfa.get_technical_efficiency()

# 2) Extrae la lista completa de betas y p-values
all_betas = np.array(sfa.get_beta())     # ej. [β0, β1, β2, β3, λ]
all_pvals = np.array(sfa.get_pvalue())   # idem

# 3) Define tus inputs (3 columnas)
input_cols = ["Bienes y servicios", "Remuneraciones", "Días Cama Disponibles"]

# 4) Selecciona únicamente los β y p-values que correspondan a tus inputs
betas = all_betas[1:1+len(input_cols)]   # toma β1, β2, β3
pvals  = all_pvals[1:1+len(input_cols)]  # toma p1, p2, p3

# 5) Construye el DataFrame alineado
df_coef = pd.DataFrame({
    "input":   input_cols,
    "beta":    betas,
    "p_value": pvals
})

# 6) Filtra significativos (p<0.05) y ordena |beta|
df_sign = df_coef[df_coef.p_value < 0.05].copy()
df_sign["abs_beta"] = df_sign.beta.abs()

if not df_sign.empty:
    var_clave = df_sign.sort_values("abs_beta", ascending=False).iloc[0].input
else:
    var_clave = "Ninguna significativa"

# 7) Calcula el resto de KPI (TE, críticos, varianza)
te_sfa      = np.array(sfa.get_technical_efficiency())
et_promedio = te_sfa.mean()
pct_crit    = (te_sfa < 0.6).mean() * 100
sigma2      = sfa.get_sigma2()

# 8) Imprime sólo los KPI
print(f"ET Promedio:      {et_promedio:.2%}")
print(f"% Hosp. críticos:  {pct_crit:.2f}%")
print(f"Variable clave:   {var_clave}")
print(f"Varianza (σ²):    {sigma2:.2f}")

# mostrar summary
print(sfa.summary())

# # print p values
# print("P-values:")
# for i, p in enumerate(sfa.get_pvalue()):
#     print(f"p{i}: {p:.4f}")

ET Promedio:      53.93%
% Hosp. críticos:  60.87%
Variable clave:   Días Cama Disponibles
Varianza (σ²):    1.09
             Parameters   Std.err  t-value  Pr(>|t|)
(Intercept)     5.24732  0.133735   39.237     0.000
x1             -0.28762  0.160117   -1.796     0.076
x2              0.27407  0.122726    2.233     0.028
x3              0.59187  0.077454    7.642     0.000
lambda          4.63395  1.549840    2.990     0.004
sigma2:  1.08873
sigmav2:  0.04845 ; sigmau2:  1.04028
log likelihood:  83.74203
None


## DEA

In [ ]:
# importar polars
import polars as pl
# probar libreria polars con la siguiente ruta "C:\Users\roesc\Desktop\hospital-efficiency-dashboard_data\df_consolidado_final_v2.csv"
# leer el archivo CSV
df = pl.read_csv("C:\\Users\\roesc\\Desktop\\hospital-efficiency-dashboard_data\\df_consolidado_final_v2.csv")
# mostrar las primeras filas del DataFrame
df.head()

In [ ]:
# pasar valores -- a 0 en columnas Bienes y servicios Remuneraciones
df = df.with_columns(
    pl.when(pl.col("Bienes y servicios") == "--").then(0).otherwise(pl.col("Bienes y servicios")).alias("Bienes y servicios"),
    pl.when(pl.col("Remuneraciones") == "--").then(0).otherwise(pl.col("Remuneraciones")).alias("Remuneraciones"),
)

# pasar columnas de tipo string a tipo float, Consultas, GRDxEgreso, Bienes y servicios, Remuneraciones, Dias Cama Disponibles, Consultas Urgencias, Examenes y Quirofanos
df = df.with_columns(
    pl.col("Bienes y servicios").str.replace(",", ".").cast(pl.Float64),
    pl.col("Remuneraciones").str.replace(",", ".").cast(pl.Float64),
)

In [ ]:
# filtrar por año 2014
df_2024 = df.filter(pl.col("Año") == 2014)

# seleccionar muestra de 25 datos
df_sample = df_2024.sample(n=25, seed=42)

import numpy as np
# ignorar filas que tengan valores 0 en Remuneraciones, Bienes y servicios, Dias Cama Disponibles y en Consultas
df = df_sample.filter(
    (pl.col("Remuneraciones") > 0) & 
    (pl.col("Bienes y servicios") > 0) & 
    (pl.col("Dias Cama Disponibles") > 0)
    & (pl.col("Consultas") > 0)
)

# contar cantidad de filas
print(f"Cantidad de filas: {df.height}")

In [ ]:
# input_cols  = ["Bienes y servicios", "Remuneraciones", "Dias Cama Disponibles"]
# output_cols = ["Consultas", "Consultas Urgencias", "GRDxEgreso", "Quirófanos"]

input_cols  = ["Bienes y servicios", "Remuneraciones", "Dias Cama Disponibles"]
output_cols = ["Consultas"]

# 6) Extrae los arrays numpy
X = df[input_cols].to_numpy()
Y = df[output_cols].to_numpy()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from Pyfrontier.frontier_model import EnvelopDEA

dea_crs = EnvelopDEA("CRS", "in")
dea_crs.fit(X, Y)

dea_crs.result[0]

import pandas as pd

# Recolectar score, slacks y lambdas de cada DMU
records_crs = []
for res in dea_crs.result:
    rec = {
        "score":      res.score,
        "slack_bs":   res.x_slack[0],   # slack en "Bienes y servicios"
        "slack_rem":  res.x_slack[1],   # slack en "Remuneraciones"
        "slack_cama": res.x_slack[2],   # slack en "Días Cama Disponibles"
        "slack_cons": res.y_slack[0],   # slack en "Consultas"
    }
    # Si quisieras alguna λ en particular, por ejemplo la j-ésima:
    # rec["lambda_5"] = res.weights[5]
    records_crs.append(rec)

df_results_crs = pd.DataFrame(records_crs, index=df["hospital_id"])
df_results_crs.head()

# calcular SE promedio 
dea_vrs = EnvelopDEA("VRS", "in")
dea_vrs.fit(X, Y)
dea_vrs.result[0]

records_vrs = []
for res in dea_vrs.result:
    rec = {
        "score":      res.score,
        "slack_bs":   res.x_slack[0],   # slack en "Bienes y servicios"
        "slack_rem":  res.x_slack[1],   # slack en "Remuneraciones"
        "slack_cama": res.x_slack[2],   # slack en "Días Cama Disponibles"
        "slack_cons": res.y_slack[0],   # slack en "Consultas"
    }
    # Si quisieras alguna λ en particular, por ejemplo la j-ésima:
    # rec["lambda_5"] = res.weights[5]
    records_vrs.append(rec)

df_results_vrs = pd.DataFrame(records_vrs, index=df["hospital_id"])
df_results_vrs.head()

df = df_results_crs.copy()

# 2) Trae los scores VRS alineados por índice (hospital_id)
#    Esto hace un left join implícito y deja NaN para los que no existan en VRS
df["score_VRS"] = df_results_vrs["score"]

# 3) Calcula SE = score_CRS / score_VRS
df["SE"] = df["score"] / df["score_VRS"]

# 4) SE promedio (ignora automáticamente los NaN)
se_promedio = df["SE"].mean()

# calcular eficiencia técnica promedio
efficiency_mean = df_results_crs["score"].mean()

# calcular % de hospitales criticos
critical_hospitals = df_results_crs[df_results_crs["score"] < 0.5]

# calcular variable de slack más alto promedio 
slack_columns = ["slack_bs", "slack_rem", "slack_cama", "slack_cons"]
df_tmp = df_results_crs[slack_columns].replace(0, np.nan)
mean_slacks = df_tmp.mean(axis=0)
var_prom_max = mean_slacks.idxmax()
val_prom_max = mean_slacks.max()

# imprimir resultados
print(f"Eficiencia técnica promedio: {efficiency_mean:.2f}")
print(f"Porcentaje de hospitales críticos: {len(critical_hospitals) / len(df_results_crs) * 100:.2f}%")
print(f"Variable de slack más alto promedio: {var_prom_max} ({val_prom_max:.2f})")
print(f"Eficiencia de escala promedio (SE): {se_promedio:.2f}")